In [ ]:
!pip install yfinance

In [ ]:
!pip install ta

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
import ta
warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import r2_score

In [67]:
from keras.models import Sequential
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Dropout
import keras

In [152]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Training Data Processing

In [167]:
df = yf.download('EURUSD=X', start='2020-01-01', end='2021-12-31')

[*********************100%***********************]  1 of 1 completed


In [168]:
df.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1.122083,1.122838,1.115947,1.122083,1.122083,0
2020-01-02,1.121894,1.122712,1.116682,1.122083,1.122083,0


In [169]:
#Add new features
df["ho"]=(((df.High-df.Open)/df.Open)*100)
df["ol"]=(((df.Open-df.Low)/df.Open)*100)
df["oc"]=(((df.Open-df.Close)/df.Open)*100)
df["hl"]=(((df.High-df.Low)/df.Low)*100)
df['rsi']=ta.momentum.rsi(df["Close"], window=14, fillna=False)
df['roc']=ta.momentum.roc(df['Close'], window = 14, fillna = False)
df['macd']=ta.trend.macd(df['Close'], window_slow=26, window_fast=12, fillna=False)
df['ema']=ta.trend.ema_indicator(df['Close'], window = 14, fillna = False)
df['sma']=ta.trend.sma_indicator(df['Close'], window = 14, fillna = False)
df['ma']=ta.momentum.kama(df["Close"], window =10, pow1=2, pow2=30, fillna=False)

In [170]:
df['next_day_return'] = np.log(df.Close) - np.log(df.Close.shift(1))

In [171]:
df = df.dropna()

In [172]:
df.shape

(498, 17)

In [173]:
df['next_day_return'] = [1 if item>0 else 0 for item in df['next_day_return']]

In [174]:
ytrain=[]
train2 = np.array(df)
for i in range(60,len(train2)):
  if(int(train2[i,16]))>0:
     ytrain.append([0,1])
  else:
    ytrain.append([1,0])
     
  #ytrain.append(int(train2[i,16])) #Adj Close is at index 4

In [175]:
df = df.drop(['next_day_return'],axis=1)

In [176]:
#Scaling the values between 0 to 1
from sklearn.preprocessing import MinMaxScaler
ss= MinMaxScaler(feature_range=(0,1))
train = ss.fit_transform(np.array(df))

In [177]:
# Feature selection
xtrain=[]

for i in range(60,len(train)):
    #xtrain.append([train[i-60:i,0],train[i-60:i,1]])
    xtrain.append(train[i-60:i,:])
    #ytrain.append(train[i,0])

xtrain, ytrain = np.array(xtrain), np.array(ytrain)

In [178]:
xtrain.shape

(438, 60, 16)

In [179]:
#Reshaping the train data to make it as input for LTSM layer input_shape(batchzise,timesteps,input_dim)
xtrain= np.reshape(xtrain,(xtrain.shape[0],xtrain.shape[2],xtrain.shape[1]))

In [180]:
xtrain.shape #Before : (463, 60, 1)

(438, 16, 60)

### Testing Data here

In [223]:
test_df = yf.download('EURUSD=X', start='2022-01-01', end='2022-06-30')

[*********************100%***********************]  1 of 1 completed


In [224]:
test_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,1.137385,1.137592,1.128541,1.137346,1.137346,0
2022-01-04,1.130199,1.132300,1.127345,1.130224,1.130224,0
2022-01-05,1.128541,1.134559,1.127764,1.128363,1.128363,0
2022-01-06,1.131222,1.133067,1.128554,1.131350,1.131350,0
2022-01-07,1.129586,1.135976,1.129050,1.129688,1.129688,0


In [225]:
#Add new features
test_df["ho"]=(((test_df.High-test_df.Open)/test_df.Open)*100)
test_df["ol"]=(((test_df.Open-test_df.Low)/test_df.Open)*100)
test_df["oc"]=(((test_df.Open-test_df.Close)/test_df.Open)*100)
test_df["hl"]=(((test_df.High-test_df.Low)/test_df.Low)*100)
test_df['rsi']=ta.momentum.rsi(test_df["Close"], window=14, fillna=False)
test_df['roc']=ta.momentum.roc(test_df['Close'], window = 14, fillna = False)
test_df['macd']=ta.trend.macd(test_df['Close'], window_slow=26, window_fast=12, fillna=False)
test_df['ema']=ta.trend.ema_indicator(test_df['Close'], window = 14, fillna = False)
test_df['sma']=ta.trend.sma_indicator(test_df['Close'], window = 14, fillna = False)
test_df['ma']=ta.momentum.kama(test_df["Close"], window =10, pow1=2, pow2=30, fillna=False)

In [226]:
test_df.shape

(129, 16)

In [227]:
test_df['next_day_return'] = np.log(test_df.Close) - np.log(test_df.Close.shift(1))

In [228]:
test = test_df.copy()

In [229]:
test = test.dropna()


In [230]:
test = test.drop(['next_day_return'],axis=1)

In [231]:
test.shape

(104, 16)

In [232]:
total= pd.concat([df,test],axis=0) # Concating train and test and then will take last 60 train point


In [233]:
total.shape

(602, 16)

In [234]:
test_input = total[len(total)-len(test)-60:].values
#test_input= test_input.reshape(-1,1) # reshaping it to get it transformed
test_input= ss.transform(test_input)

In [235]:
xtest= []
for i in range(60,160):
    xtest.append(test_input[i-60:i,:]) #creating input for  prediction

In [236]:
xtest= np.array(xtest)

In [237]:
xtest.shape

(100, 60, 16)

In [238]:
xtest= np.reshape(xtest,(xtest.shape[0],xtest.shape[2],xtest.shape[1]))

In [239]:
test_df['next_day_return'] = [1 if item>0 else 0 for item in test_df['next_day_return']]

In [248]:
test_actual = np.array(test_df[0:100]['next_day_return'])
ytest = []
for i in range(0,len(test_actual)):
  if(int(test_actual[i]))>0:
     ytest.append([0,1])
  else:
    ytest.append([1,0])

In [249]:
test_actual.shape

(100,)

### Build an GRU Model

In [252]:
#initialisizng the model 
model = Sequential()

model.add(GRU(units=50,return_sequences=True,kernel_initializer='glorot_uniform',input_shape=(xtrain.shape[1],xtrain.shape[2])))
model.add(Dropout(0.2))

model.add(GRU(units=50,kernel_initializer='glorot_uniform',return_sequences=True))
model.add(Dropout(0.2))

model.add(GRU(units=50,kernel_initializer='glorot_uniform',return_sequences=True))
model.add(Dropout(0.2))

model.add(GRU(units=50,kernel_initializer='glorot_uniform'))
model.add(Dropout(0.2))

model.add(Dense(2,activation='sigmoid'))

model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics='accuracy')

#fitting the network
model.fit(xtrain,ytrain,batch_size=30,epochs=100)

Epoch 1/100
15/15 [==============================] - 10s 40ms/step - loss: 0.6977 - accuracy: 0.5205
Epoch 2/100
15/15 [==============================] - 1s 40ms/step - loss: 0.6987 - accuracy: 0.4977
Epoch 3/100
15/15 [==============================] - 1s 41ms/step - loss: 0.6994 - accuracy: 0.5137
Epoch 4/100
15/15 [==============================] - 1s 40ms/step - loss: 0.6962 - accuracy: 0.5023
Epoch 5/100
15/15 [==============================] - 1s 41ms/step - loss: 0.6944 - accuracy: 0.5046
Epoch 6/100
15/15 [==============================] - 1s 40ms/step - loss: 0.6942 - accuracy: 0.5091
Epoch 7/100
15/15 [==============================] - 1s 40ms/step - loss: 0.6966 - accuracy: 0.4909
Epoch 8/100
15/15 [==============================] - 1s 43ms/step - loss: 0.6952 - accuracy: 0.5183
Epoch 9/100
15/15 [==============================] - 1s 40ms/step - loss: 0.6935 - accuracy: 0.5183
Epoch 10/100
15/15 [==============================] - 1s 43ms/step - loss: 0.6929 - accuracy: 0.516

In [253]:
predicted_value= model.predict(xtest)


In [254]:
predicted_classes = predicted_value.argmax(axis=1)
predicted_classes

array([1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0])

In [255]:

confusion_matrix(test_actual, predicted_classes)

array([[26, 33],
       [16, 25]])

In [256]:

accuracy_score(test_actual, predicted_classes)

0.51